Source:
https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset

Predict the number of bikes rented.

In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb

Load dataset.

In [5]:
file_name = "hour.csv"
data = pd.read_csv(file_name)
data = pd.DataFrame(data)
data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Define train and test datasets.

In [12]:
X = data.drop(['cnt','instant','dteday','casual','registered'], axis=1)
y = data["cnt"]

# Split into training and test set
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train.reindex(X_train.index)], axis=1)
test_data = pd.concat([X_test, y_test.reindex(X_test.index)], axis=1)

Built XGBoost models for train and test sets.

In [31]:
from sklearn.metrics import mean_absolute_error
reg = xgb.XGBRegressor(
    tree_method="hist",
    eval_metric=mean_absolute_error,
)
reg.fit(X_train, y_train)
score = reg.score(X_train, y_train)  
print("Training score: ", score)

Training score:  0.9759218516447414


In [32]:
scores = cross_val_score(reg, X_train, y_train,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: 0.95


So we have about 95 percent accuracy for the model.

Now save the model.

In [34]:
import pickle
filename1 = 'finalized_model_xgb.sav'
model = pickle.dump(reg, open(filename1, 'wb'))

Load them back in. Fit to test data and check score.

In [35]:
#XGBoost
loaded_model_xgb = pickle.load(open(filename1, 'rb'))
result_xgb = loaded_model_xgb.score(X_test, y_test)
print(result_xgb)

0.9486919011895639


Same 95 percent accuracy for the model.